# PostgreSQL 16.6+

## 0) 前提

* エンジン: **PostgreSQL 16.6+**
* 並び順: 任意
* `NOT IN` 回避（`EXISTS` / `LEFT JOIN ... IS NULL` を推奨）
* 判定は ID 基準、表示は仕様どおり（`customer_id`）

---

## 1) 問題

* `{{PROBLEM_STATEMENT}}`
  Product テーブルに存在する **全ての `product_key`** を購入した `customer_id` を求める
* 入力: `{{TABLES_OR_SCHEMAS}}`
  `Customer(customer_id int, product_key int)`（重複あり）
  `Product(product_key int)`（主キー）
* 出力: `{{OUTPUT_COLUMNS_AND_RULES}}`
  列: `customer_id` のみ、順序任意、重複なし

---

## 2) 最適解（単一クエリ）

> 先に `Customer` の重複を除去し、**`Product` に存在するキーだけ**に絞ってから顧客ごと件数を集計。全 `Product` 件数との一致で判定します。PostgreSQL では CTE を使い、総数は一度だけ評価します。

```sql
WITH uniq AS (  -- 顧客×商品のユニーク集合
  SELECT DISTINCT customer_id, product_key
  FROM Customer
  WHERE product_key IS NOT NULL
),
cp AS (         -- Product に存在する key のみに限定
  SELECT u.customer_id, u.product_key
  FROM uniq u
  JOIN Product p USING (product_key)
),
prod AS (       -- Product 総数（1回だけ計算）
  SELECT COUNT(*) AS total_products
  FROM Product
),
win AS (        -- 顧客ごとの購入ユニーク数
  SELECT
    customer_id,
    COUNT(*) AS bought_cnt
  FROM cp
  GROUP BY customer_id
)
SELECT w.customer_id
FROM win w
CROSS JOIN prod pr
WHERE w.bought_cnt = pr.total_products;

Runtime 513 ms
Beats 54.78%

```

* ポイント

  * **正しさ**: `Product` に無い `product_key` を数えない（`JOIN Product` 済み）
  * **効率**: `prod` は 1 回だけ評価、`uniq` で重複除去→集約を軽量化

### 代替（関係除算の典型：二重 `NOT EXISTS`）

```sql
SELECT DISTINCT c.customer_id
FROM Customer c
WHERE NOT EXISTS (
  SELECT 1
  FROM Product p
  WHERE NOT EXISTS (
    SELECT 1
    FROM Customer c2
    WHERE c2.customer_id = c.customer_id
      AND c2.product_key = p.product_key
  )
);

Runtime 1872 ms
Beats 5.06%

```

> 読みやすいが、データ量次第でネスト `EXISTS` が重くなりやすい。上の集約比較の方が安定。

---

## 3) 要点解説

* **重複/NULL**: `Customer` 側の重複は **`DISTINCT`** で除去。`product_key IS NOT NULL` を明示。
* **除算の定石**:

  1. `JOIN Product` で対象を「存在する key」のみに限定
  2. 顧客ごとに **ユニーク購入数** を集計
  3. **全商品数と一致**で判定
* **ウィンドウ不要**: 本件は単純な集合比較のため、ウィンドウ関数なしが最短・高速。

---

## 4) 計算量（概算）

* `uniq` の重複除去: **O(N log N)**（`(customer_id, product_key)` で実質線形近似）
* `cp` の結合: **O(N)** 近似（`Product(product_key)` がPKのため）
* 顧客集約: **O(N log N)**（ハッシュ集約で線形近似）

---

## 5) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[Customer 入力] --> B[重複除去 DISTINCT]
  C[Product 入力]  --> D[Product 総数カウント]
  B --> E[Product に存在する key へ JOIN]
  E --> F[顧客ごとユニーク数を集計]
  D --> G[全商品数と一致判定]
  F --> G
  G --> H[出力 customer_id]
```

---

### 実運用メモ（推奨インデックス）

```sql
-- Product は PK で OK
CREATE INDEX CONCURRENTLY IF NOT EXISTS idx_customer_cust_prod
  ON Customer (customer_id, product_key);
CREATE INDEX CONCURRENTLY IF NOT EXISTS idx_customer_prod_cust
  ON Customer (product_key, customer_id);
```

* ユニーク化・結合・集約の全パスをカバーし、実行計画の自由度を高めます。

この結果を見る限り、CTE版はもう十分に「正しい & 使える」水準ですが、まだチューニング余地があります。主に3点です。

---

## 1. CTEをインライン化してプランナーに最適化させる

PostgreSQLは`WITH`を「最適化バリア」にしないようになってきていますが、状況次第では依然としてCTEがマテリアライズ扱いされることがあり、そのせいで余計なスキャンが発生するケースがあります。
`uniq -> cp -> win` は合成可能なので、単一クエリに潰してプランナーに自由度を与えます。

```sql
SELECT x.customer_id
FROM (
  SELECT
    c.customer_id,
    COUNT(DISTINCT c.product_key) AS bought_cnt
  FROM Customer c
  JOIN Product p
    ON p.product_key = c.product_key    -- Productに存在するkeyだけ数える
  WHERE c.product_key IS NOT NULL       -- NULL防衛
  GROUP BY c.customer_id
) AS x
CROSS JOIN (
  SELECT COUNT(*) AS total_products
  FROM Product
) AS pr
WHERE x.bought_cnt = pr.total_products;

Runtime 502 ms
Beats 62.73%

```

### これで狙えること

* `uniq` の `SELECT DISTINCT` と `cp` の結合を、PostgreSQLにまとめて最適化させる
* `COUNT(DISTINCT ...)` によって「顧客ごとのユニーク商品数」を一気に出す
  → 手動で一時テーブルを積まない分、プランがシンプルになる

実際、この形は論理的にあなたのCTE版と同じ意味を保っています（`uniq`の役割= `COUNT(DISTINCT ...)` で内包）。

### 注意

* `COUNT(DISTINCT c.product_key)` は、データ量が非常に大きい場合にハッシュ集約のコストが支配する可能性があります。ただし典型的なLeetCode/業務レベルの件数ではまずボトルネックにならず、CTE分割より速いことが多いです。

---

## 2. 不要なCROSS JOINを外し、定数として比較する（さらに単純化）

PostgreSQLはスカラサブクエリをかなりうまくキャッシュします。`CROSS JOIN` 自体は悪ではないですが、読みやすさ＆最適化計画の自由度の観点で、最終比較はスカラサブクエリに戻してしまっても問題ありません。

```sql
SELECT
  c.customer_id
FROM Customer c
JOIN Product p
  ON p.product_key = c.product_key
WHERE c.product_key IS NOT NULL
GROUP BY c.customer_id
HAVING COUNT(DISTINCT c.product_key) = (
  SELECT COUNT(*) FROM Product
);

Runtime 595 ms
Beats 21.28%

```

メリット:

* 読む側に優しい
* 実行計画も単純化（1回の集約＋スカラサブクエリ）
* 多段CTEよりインラインhash aggの最適化を受けやすい

実務ではこの1本でOKにすることが多いです。LeetCode系でも通るパターン。

---

## 3. インデックス設計で 513ms → さらに圧縮

論理式をいくら磨いても、I/Oボトルネックが太いままだと頭打ちです。今回のWHERE・JOIN・GROUP BY・COUNT(DISTINCT)に効くインデックスを貼ると、劇的に走査量が下がります。

おすすめは2本。

```sql
-- 1. 顧客→商品 方向
CREATE INDEX CONCURRENTLY IF NOT EXISTS idx_customer_customer_product
  ON Customer (customer_id, product_key);

-- 2. 商品→顧客 方向（JOIN用）
CREATE INDEX CONCURRENTLY IF NOT EXISTS idx_customer_product_customer
  ON Customer (product_key, customer_id);
```

なぜ2本？

* `JOIN Product p ON p.product_key = c.product_key` では `Customer.product_key` での絞り込み・ルックアップが効く
* `GROUP BY c.customer_id` ＋ `COUNT(DISTINCT c.product_key)` では `(customer_id, product_key)` の順序が効く

  * PostgreSQLは`COUNT(DISTINCT ...)`を内部的にソート/ハッシュするので、`(customer_id, product_key)`の順のインデックスは「同じ顧客の商品のまとまり」を局所化しやすい
  * つまりメモリに優しい

`Product.product_key` はPK前提でもう張られているはずなのでそのままでOK。

これを張るだけで「winテーブル集約」みたいな手順を作らなくても、プランナーがIndex Scan/Hash Aggregateを組んでくれるようになります。
特にデータが

* Customer行数 >> Product行数
* 特定のcustomer_idに大量のproduct_keyがある（ヘビーユーザー）
  という分布だと効果が大きいです。

---

## まとめ（どこを変えるべき？）

* **まずやるべき最適化**
  下記の最小版に置き換えること。CTEを潰し、`COUNT(DISTINCT)`に寄せ、比較はスカラサブクエリでOK。

  ```sql
  SELECT
    c.customer_id
  FROM Customer c
  JOIN Product p
    ON p.product_key = c.product_key
  WHERE c.product_key IS NOT NULL
  GROUP BY c.customer_id
  HAVING COUNT(DISTINCT c.product_key) = (
    SELECT COUNT(*) FROM Product
  );
  ```

* **プラスαでやるべき最適化**
  `Customer(customer_id, product_key)` と `Customer(product_key, customer_id)` の2本の複合インデックスを貼る。
  これが実行時間を一番押し下げる現実的な手段。

* **しなくていいこと**
  二重 `NOT EXISTS` 版はTLE気味（1872ms）で、これはもう「遅いけど正しいリファレンス実装」という位置づけでOK。さらに最適化する価値は低い。

つまり、ロジック面はもう合格。今はプランナーに寄せた形にする＋インデックスの用意。この2点で 513ms からまだ落とす余地はあります。

